<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250723/%E3%82%AB%E3%83%83%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0%E3%82%B9%E3%83%88%E3%83%83%E3%82%AF%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#カッティングストック問題

## 準備
最初に、問題を解くために必要なライブラリ `mip` をインストールする。


In [ ]:
%%capture
# ライブラリをインストールします
!pip install mip

-----

## 演習問題 1

以下の条件でカッティングストック問題を解き、最小原反総数を求めよ。

  * **原反の幅**: $L=200 \text{cm}$
  * **製品データ**:
      * 製品A: 幅 $w\_A = 70 \\text{cm}$, 需要 $D\_A = 15$ 本
      * 製品B: 幅 $w\_B = 50 \\text{cm}$, 需要 $D\_B = 20$ 本
      * 製品C: 幅 $w\_C = 30 \\text{cm}$, 需要 $D\_C = 25$ 本
  * **定義済みカッティングパターン** (製品[A, B, C]の取得本数):
      * パターン 0: `[2, 1, 0]` (幅: $70 \\times 2 + 50 \\times 1 = 190 \\text{cm}$)
      * パターン 1: `[1, 2, 1]` (幅: $70 \\times 1 + 50 \\times 2 + 30 \\times 1 = 200 \\text{cm}$)
      * パターン 2: `[1, 1, 2]` (幅: $70 \\times 1 + 50 \\times 1 + 30 \\times 2 = 180 \\text{cm}$)
      * パターン 3: `[0, 4, 0]` (幅: $50 \\times 4 = 200 \\text{cm}$)
      * パターン 4: `[0, 2, 3]` (幅: $50 \\times 2 + 30 \\times 3 = 190 \\text{cm}$)
      * パターン 5: `[0, 0, 6]` (幅: $30 \\times 6 = 180 \\text{cm}$)
      * パターン 6: `[0, 1, 5]` (幅: $50 \\times 1 + 30 \\times 5 = 200 \\text{cm}$)

###数理モデルの定式化

この問題を整数計画問題として定式化します。

  * **決定変数**:

      *
$x_j$: パターン $j$
$(j \in \{0, 1, ..., 6\})$ を使用する回数（原反の本数）。$x_j$ は非負の整数。

  * **目的関数**:

      * 使用する原反の総本数を最小化する。
        $$\min \sum_{j=0}^{6} x_j$$

  * **制約条件**:

      * 各製品の需要量を満たす必要がある。
          * 製品A: $2x\_0 + 1x\_1 + 1x\_2 \\ge 15$
          * 製品B: $1x\_0 + 2x\_1 + 1x\_2 + 4x\_3 + 2x\_4 + 1x_6\\ge 20$
          * 製品C: $1x\_1 + 2x\_2 + 3x\_4 + 6x\_5 + 5x_6 \\ge 25$
      * 変数は非負の整数でなければならない。
          *
$x_j \ge 0$ and integer for $j \in \{0, 1, ..., 6\}$

###Python (MIP) による実装


In [ ]:
import mip
import numpy as np

# --- 問題データの設定 ---
# 原反の幅
stock_width = 200

# 製品データ
product_names = ["A (70cm)", "B (50cm)", "C (30cm)"]
product_widths = np.array([70, 50, 30])
demands = np.array([15, 20, 25])
num_product_types = len(product_names)

# 事前に定義されたカッティングパターン
# 各行が1つのパターンを表し、列が製品A, B, Cの取得本数を示す
defined_patterns = [
    [2, 1, 0],  # パターン 0
    [1, 2, 1],  # パターン 1
    [1, 1, 2],  # パターン 2
    [0, 4, 0],  # パターン 3
    [0, 2, 3],  # パターン 4
    [0, 0, 6],  # パターン 5
    [0, 1, 5],  # パターン 6
]
pattern_yields = np.array(defined_patterns)
num_defined_patterns = len(defined_patterns)

# --- 数理モデルの構築 ---
# 1. モデルの作成
model = mip.Model(name="exercise_1", sense=mip.MINIMIZE)

# 2. 変数の定義
# x[j]: パターンjを使用する回数 (原反の本数)
x = [model.add_var(var_type=mip.INTEGER, lb=0, name=f"x_{j}") for j in range(num_defined_patterns)]

# 3. 目的関数の設定
# 使用する原反の総本数を最小化
model.objective = mip.xsum(x[j] for j in range(num_defined_patterns))

# 4. 制約条件の追加
# 需要充足制約
for i in range(num_product_types):
    # 各製品iについて、生産される総本数が需要量以上であること
    model += mip.xsum(pattern_yields[j][i] * x[j] for j in range(num_defined_patterns)) >= demands[i], f"demand_{product_names[i]}"

# --- 問題の求解と結果の表示 ---
# 5. 問題の求解
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"使用する原反の最小総本数: {int(model.objective_value)}")
    print("\n各パターンの使用回数:")
    total_production = np.zeros(num_product_types)
    for j in range(num_defined_patterns):
        if x[j].x > 1e-6: # ほぼ0より大きい場合
            print(f"  パターン {j} (製品構成: {pattern_yields[j]}): {int(round(x[j].x))} 回")
            total_production += pattern_yields[j] * int(round(x[j].x))

    print("\n各製品の総生産量と需要量:")
    for i in range(num_product_types):
        print(f"  {product_names[i]}: 生産量 {int(total_production[i])}, 需要量 {demands[i]} (超過: {int(total_production[i] - demands[i])})")

elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解が存在しません。")
else:
    print(f"最適化ステータス: {status}")

使用する原反の最小総本数: 15

各パターンの使用回数:
  パターン 0 (製品構成: [2 1 0]): 5 回
  パターン 1 (製品構成: [1 2 1]): 6 回
  パターン 6 (製品構成: [0 1 5]): 4 回

各製品の総生産量と需要量:
  A (70cm): 生産量 16, 需要量 15 (超過: 1)
  B (50cm): 生産量 21, 需要量 20 (超過: 1)
  C (30cm): 生産量 26, 需要量 25 (超過: 1)


-----

## 演習問題 2

演習問題1の条件のうち、製品の需要量のみが以下のように変更された場合の問題を解き、最小原反総数を求めよ。

  * **製品データ (需要量のみ変更)**:
      * 製品A: 幅 70cm, 需要 **8本**
      * 製品B: 幅 50cm, 需要 **25本**
      * 製品C: 幅 30cm, 需要 **15本**
  * 原反幅、製品幅、カッティングパターンは演習問題1と同じである。

###数理モデルの定式化

数理モデルの構造は演習問題1と同じですが、制約条件の右辺である需要量の値が変更されます。

  * **決定変数**と**目的関数**は演習問題1と同様。

  * **制約条件 (需要充足制約のみ変更)**:

      * 製品A:
$2x_0 + 1x_1 + 1x_2 \ge 8$
      * 製品B:
$1x_0 + 2x_1 + 1x_2 + 4x_3 + 2x_4 + 1x_6 \ge 25$
      * 製品C:
$1x_1 + 2x_2 + 3x_4 + 6x_5 + 5x_6 \ge 15$

###Python (MIP) による実装


In [ ]:
import mip
import numpy as np

# --- 問題データの設定 ---
# 原反の幅
stock_width = 200

# 製品データ
product_names = ["A (70cm)", "B (50cm)", "C (30cm)"]
product_widths = np.array([70, 50, 30])
# 需要量を変更
demands = np.array([8, 25, 15]) # <- 変更点
num_product_types = len(product_names)

# 事前に定義されたカッティングパターン (演習問題1と同じ)
defined_patterns = [
    [2, 1, 0],  # パターン 0
    [1, 2, 1],  # パターン 1
    [1, 1, 2],  # パターン 2
    [0, 4, 0],  # パターン 3
    [0, 2, 3],  # パターン 4
    [0, 0, 6],  # パターン 5
    [0, 1, 5],  # パターン 6
]
pattern_yields = np.array(defined_patterns)
num_defined_patterns = len(defined_patterns)

# --- 数理モデルの構築 ---
model = mip.Model(name="exercise_2", sense=mip.MINIMIZE)
x = [model.add_var(var_type=mip.INTEGER, lb=0, name=f"x_{j}") for j in range(num_defined_patterns)]
model.objective = mip.xsum(x[j] for j in range(num_defined_patterns))
for i in range(num_product_types):
    model += mip.xsum(pattern_yields[j][i] * x[j] for j in range(num_defined_patterns)) >= demands[i], f"demand_{product_names[i]}"

# --- 問題の求解と結果の表示 ---
status = model.optimize()
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"使用する原反の最小総本数: {int(model.objective_value)}")
    print("\n各パターンの使用回数:")
    total_production = np.zeros(num_product_types)
    for j in range(num_defined_patterns):
        if x[j].x > 1e-6:
            print(f"  パターン {j} (製品構成: {pattern_yields[j]}): {int(round(x[j].x))} 回")
            total_production += pattern_yields[j] * int(round(x[j].x))

    print("\n各製品の総生産量と需要量:")
    for i in range(num_product_types):
        print(f"  {product_names[i]}: 生産量 {int(total_production[i])}, 需要量 {demands[i]} (超過: {int(total_production[i] - demands[i])})")
else:
    print(f"最適化ステータス: {status}")

使用する原反の最小総本数: 12

各パターンの使用回数:
  パターン 1 (製品構成: [1 2 1]): 8 回
  パターン 3 (製品構成: [0 4 0]): 2 回
  パターン 6 (製品構成: [0 1 5]): 2 回

各製品の総生産量と需要量:
  A (70cm): 生産量 8, 需要量 8 (超過: 0)
  B (50cm): 生産量 26, 需要量 25 (超過: 1)
  C (30cm): 生産量 18, 需要量 15 (超過: 3)


-----

## 演習問題3

演習問題1の条件において、「1本の原反から生じるトリムロスが0cmを超えるパターンは使用してはならない」という制約が追加された問題を解け。（まず、各パターンのトリムロスを計算し、使用可能なパターンを特定する必要がある。）

  * **追加制約**: トリムロス $=0\text{cm}$ のパターンのみ使用可能。
  * 原反幅 $L=200\text{cm}$
  * その他の条件は演習問題1と同じである。

###数理モデルの定式化

この問題では、まず前処理として各パターンのトリムロスを計算し、制約を満たすパターンのみをモデルに含めます。

  * **前処理：使用可能パターンの特定**
      * 原反幅 $L=200$
      * 製品幅: $w\_A=70, w\_B=50, w\_C=30$
      * P0 `[2,1,0]`: 幅 $190$, トリムロス $10$  ($>0$ なので**使用不可**)
      * P1 `[1,2,1]`: 幅 $200$, トリムロス $0$   (**使用可**)
      * P2 `[1,1,2]`: 幅 $180$, トリムロス $20$  ($>0$ なので**使用不可**)
      * P3 `[0,4,0]`: 幅 $200$, トリムロス $0$   (**使用可**)
      * P4 `[0,2,3]`: 幅 $190$, トリムロス $10$  ($>5$ なので**使用不可**)
      * P5 `[0,0,6]`: 幅 $180$, トリムロス $20$  ($>5$ なので**使用不可**)
      * P6 `[0,1,5]`: 幅 $200$, トリムロス $0$   (**使用可**)
  * **使用可能なパターン**: パターン1, 3, 6 のみ。

この情報に基づき、使用可能なパターンだけで数理モデルを構築します。定式化の構造は演習問題1と同様ですが、考慮する変数が $x_1, x_3, x_6$ のみとなります。

###Python (MIP) による実装


In [ ]:
import mip
import numpy as np

# --- 問題データの設定 (演習問題1と同じ) ---
stock_width = 200
product_names = ["A (70cm)", "B (50cm)", "C (30cm)"]
product_widths = np.array([70, 50, 30])
demands = np.array([15, 20, 25])
num_product_types = len(product_names)
all_patterns = np.array([
    [2, 1, 0], [1, 2, 1], [1, 1, 2], [0, 4, 0],
    [0, 2, 3], [0, 0, 6], [0, 1, 5]
])

# --- 前処理: 使用可能なパターンをフィルタリング ---
pattern_total_widths = np.dot(all_patterns, product_widths)
pattern_trim_losses = stock_width - pattern_total_widths

# トリムロスが5cm以下のパターンのみを抽出
usable_pattern_indices = [i for i, loss in enumerate(pattern_trim_losses) if loss <= 5]
pattern_yields = all_patterns[usable_pattern_indices]
num_defined_patterns = len(pattern_yields)

print("--- 使用可能なパターン ---")
for i, original_idx in enumerate(usable_pattern_indices):
    print(f"元のパターン {original_idx} (構成: {pattern_yields[i]}, トリムロス: {pattern_trim_losses[original_idx]})")
print("-" * 25)

# --- 数理モデルの構築 ---
model = mip.Model(name="exercise_5", sense=mip.MINIMIZE)
x = [model.add_var(var_type=mip.INTEGER, lb=0, name=f"x_{usable_pattern_indices[j]}") for j in range(num_defined_patterns)]
model.objective = mip.xsum(x[j] for j in range(num_defined_patterns))
for i in range(num_product_types):
    model += mip.xsum(pattern_yields[j][i] * x[j] for j in range(num_defined_patterns)) >= demands[i], f"demand_{product_names[i]}"

# --- 問題の求解と結果の表示 ---
status = model.optimize()
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"使用する原反の最小総本数: {int(model.objective_value)}")
    print("\n各パターンの使用回数:")
    total_production = np.zeros(num_product_types)
    for j in range(num_defined_patterns):
        if x[j].x > 1e-6:
            original_idx = usable_pattern_indices[j]
            print(f"  パターン {original_idx} (製品構成: {pattern_yields[j]}): {int(round(x[j].x))} 回")
            total_production += pattern_yields[j] * int(round(x[j].x))

    print("\n各製品の総生産量と需要量:")
    for i in range(num_product_types):
        print(f"  {product_names[i]}: 生産量 {int(total_production[i])}, 需要量 {demands[i]} (超過: {int(total_production[i] - demands[i])})")
elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解が存在しません。使用可能なパターンでは需要を満たせません。")
else:
    print(f"最適化ステータス: {status}")

--- 使用可能なパターン ---
元のパターン 1 (構成: [1 2 1], トリムロス: 0)
元のパターン 3 (構成: [0 4 0], トリムロス: 0)
元のパターン 6 (構成: [0 1 5], トリムロス: 0)
-------------------------
最適解が見つかりました！
使用する原反の最小総本数: 17

各パターンの使用回数:
  パターン 1 (製品構成: [1 2 1]): 15 回
  パターン 6 (製品構成: [0 1 5]): 2 回

各製品の総生産量と需要量:
  A (70cm): 生産量 15, 需要量 15 (超過: 0)
  B (50cm): 生産量 32, 需要量 20 (超過: 12)
  C (30cm): 生産量 25, 需要量 25 (超過: 0)
